This notebook generates the following panels:
- ( ) Summary - Raster: PSTH average of all depth neurons, sorted by fitted preferred depth

use env 2p_analysis_cottage

In [ ]:
VERSION = 1

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import defopt
import pickle
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # save text as text not outlines
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import flexiznam as flz
from cottage_analysis.depth_analysis.filepath.generate_filepaths import *
from cottage_analysis.imaging.common import find_frames
from cottage_analysis.imaging.common import imaging_loggers_formatting as format_loggers
from cottage_analysis.stimulus_structure import sphere_structure as vis_stim_structure
from cottage_analysis.depth_analysis.plotting.plotting_utils import *
from cottage_analysis.depth_analysis.plotting.basic_vis_plots import *
from cottage_analysis.depth_analysis.depth_preprocess.process_params import *
from cottage_analysis.depth_analysis.depth_preprocess.process_trace import *

In [ ]:
save_root = '/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver'+str(VERSION)+'/close_raster_summary/'
if not os.path.exists(save_root):
    os.makedirs(save_root)
    
rawdata_root = flz.PARAMETERS['data_root']['raw']
root = flz.PARAMETERS['data_root']['processed']

project='hey2_3d-vision_foodres_20220101'
mouses = ['PZAH6.4b','PZAG3.4f']
V1_sessions = [['S20220512','S20220511','S20220429','S20220419','S20220426','S20220506','S20220503','S20220505','S20220516'],['S20220504','S20220426','S20220421','S20220419','S20220505','S20220503','S20220510','S20220512','S20220511']]
protocol='SpheresPermTubeReward'
playback=False
flm_sess = flz.get_flexilims_session(project_id=project)

# COMBINE PSTH FROM ALL DEPTH NEURONS OF ALL SESSIONS INTO A GIANT LIST
all_neurons_psth = []
all_neurons_preferred_depths = []
for k in range(len(V1_sessions)):
    mouse = mouses[k]
    for session in V1_sessions[k]:
        print(mouse, session)
        depth_list = [0.06, 0.19, 0.6, 1.9, 6]
        choose_trials = 50
        frame_rate = 15
        speed_thr_cal = 0.2  # m/s, threshold for running speed when calculating depth neurons
        speed_thr = 0.01  #m/s
        manual_choose_rois = True
        manually_chosen_rois = [1]
        distance_max = 6

        # Generate file path
        original_files = {}
        original_files['rawdata_folder'], \
        original_files['protocol_folder'], \
        original_files['analysis_folder'], \
        original_files['suite2p_folder'], \
        original_files['trace_folder'] = generate_filepaths.generate_file_folders(
            project=project,
            mouse=mouse,
            session=session,
            protocol=protocol,
            flexilims_session=flm_sess)

        if playback:
            playback_files = {}
            playback_files['rawdata_folder'], \
            playback_files['protocol_folder'], \
            playback_files['analysis_folder'], \
            playback_files['suite2p_folder'], \
            playback_files['trace_folder'] = generate_filepaths.generate_file_folders(
                project=project,
                mouse=mouse,
                session=session,
                protocol=protocol+'Playback',
                flexilims_session=flm_sess)

        # Load files
        # Load suite2p files
        ops = np.load(original_files['suite2p_folder']/'ops.npy', allow_pickle=True)
        ops = ops.item()
        iscell = np.load(original_files['suite2p_folder']/'iscell.npy', allow_pickle=True)[:, 0]
        F = np.load(original_files['trace_folder']/'F.npy', allow_pickle=True)
        # Fast = np.load(trace_folder + 'Fast.npy', allow_pickle=True)
        # Fneu = np.load(trace_folder + 'Fneu.npy', allow_pickle=True)
        # spks = np.load(trace_folder + 'spks.npy', allow_pickle=True)

        # All_rois
        which_rois = (np.arange(F.shape[0]))[iscell.astype('bool')]

        # dffs_ast
        dffs_original = np.load(original_files['trace_folder']/'dffs_ast.npy')
        if playback:
            dffs_playback = np.load(playback_files['trace_folder']/'dffs_ast.npy')

        # Load stim structure file
        with open(original_files['protocol_folder']/'img_VS.pickle', 'rb') as handle:
            img_VS_original = pickle.load(handle)
        if os.path.exists(original_files['protocol_folder']/'stim_dict.pickle'):
            with open(original_files['protocol_folder']/'stim_dict.pickle', 'rb') as handle:
                stim_dict_original = pickle.load(handle)
        else:
            stim_dict_original = create_stim_dict(depth_list=depth_list, img_VS=img_VS_original, choose_trials=choose_trials)
        with open(original_files['analysis_folder']/'plane0/gaussian_depth_tuning_fit_new_0.5.pickle', 'rb') as handle:
            gaussian_depth_original = pickle.load(handle)

        if playback:
            with open(playback_files['protocol_folder']/'img_VS.pickle', 'rb') as handle:
                img_VS_playback = pickle.load(handle)
            if os.path.exists(playback_files['protocol_folder']/'stim_dict.pickle'):
                with open(playback_files['protocol_folder']/'stim_dict.pickle', 'rb') as handle:
                    stim_dict_playback = pickle.load(handle)
            else:
                stim_dict_playback = create_stim_dict(depth_list=depth_list, img_VS=img_VS_playback, choose_trials=choose_trials)
            with open(playback_files['analysis_folder']/'plane0/gaussian_depth_tuning_fit_new_0.5.pickle', 'rb') as handle:
                gaussian_depth_playback = pickle.load(handle)

        # Load depth neurons
        depth_neurons = np.load(original_files['analysis_folder']/'plane0/depth_neurons.npy')
        max_depths_original = np.load(original_files['analysis_folder']/'plane0/max_depths_index.npy')
        # max_depths_gaussian_fit = np.load(original_files['analysis_folder'] + 'plane0/max_LogDepth_gaussianfit.npy')
        if playback:
            max_depths_playback = np.load(playback_files['analysis_folder']/'plane0/max_depths_index.npy')
            # max_depths_gaussian_fit = np.load(playback_files['analysis_folder'] + 'plane0/max_LogDepth_gaussianfit.npy')

        # PARAMS PROCESSING
        # Speed array
        # Running speed is thresholded with a small threshold to get rid of non-zero values (default threshold 0.01)
        # Closeloop
        speeds_original = img_VS_original.MouseZ.diff() / img_VS_original.HarpTime.diff() # with no playback. EyeZ and MouseZ should be the same.
        speeds_original[0] = 0
        speeds_original = thr(speeds_original, speed_thr)
        speed_arr_original, _ = create_speed_arr(speeds_original, depth_list, stim_dict_original, mode='sort_by_depth', protocol='fix_length',
                                        blank_period=0, frame_rate=frame_rate)
        speed_arr_mean_original = np.nanmean(speed_arr_original,axis=1)
        speed_arr_noblank_original,_ = create_speed_arr(speeds_original, depth_list, stim_dict_original, mode='sort_by_depth', protocol='fix_length', blank_period=0, frame_rate=frame_rate)
        speed_arr_noblank_mean_original = np.nanmean(speed_arr_noblank_original,axis=1)
        speed_arr_blank_original,_ = create_speed_arr(speeds_original, depth_list, stim_dict_original, mode='sort_by_depth', protocol='fix_length', isStim=False, blank_period=0, frame_rate=frame_rate)
        frame_num_pertrial_max_original = speed_arr_noblank_original.shape[2]
        total_trials_original = speed_arr_noblank_original.shape[1]
        optics_original = calculate_OF(rs=speeds_original, img_VS=img_VS_original, mode='no_RF')

        # Array of distance travelled from each trial start (m)
        distance_arr_original, _ = create_speed_arr(img_VS_original['EyeZ'], depth_list, stim_dict_original, mode='sort_by_depth',
                                            protocol='fix_length', blank_period=0, frame_rate=frame_rate)
        for idepth in range(distance_arr_original.shape[0]):
            for itrial in range(distance_arr_original.shape[1]):
                distance_arr_original[idepth, itrial, :] = distance_arr_original[idepth, itrial, :] - distance_arr_original[idepth, itrial, 0]

        # Openloop
        if playback:
            speeds_playback = img_VS_playback.MouseZ.diff() / img_VS_playback.HarpTime.diff() # with no playback. EyeZ and MouseZ should be the same.
            speeds_playback[0] = 0
            speeds_playback = thr(speeds_playback, speed_thr)
            speed_arr_playback, _ = create_speed_arr(speeds_playback, depth_list, stim_dict_playback, mode='sort_by_depth', protocol='fix_length',
                                            blank_period=0, frame_rate=frame_rate)
            speed_arr_mean_playback = np.nanmean(speed_arr_playback,axis=1)
            speed_arr_noblank_playback,_ = create_speed_arr(speeds_playback, depth_list, stim_dict_playback, mode='sort_by_depth', protocol='fix_length', blank_period=0, frame_rate=frame_rate)
            speed_arr_noblank_mean_playback = np.nanmean(speed_arr_noblank_playback,axis=1)
            speed_arr_blank_playback,_ = create_speed_arr(speeds_playback, depth_list, stim_dict_playback, mode='sort_by_depth', protocol='fix_length', isStim=False, blank_period=0, frame_rate=frame_rate)
            frame_num_pertrial_max_playback = speed_arr_noblank_playback.shape[2]
            total_trials_playback = speed_arr_noblank_playback.shape[1]

            speeds_eye_playback = img_VS_playback.EyeZ.diff() / img_VS_playback.HarpTime.diff()  # EyeZ is how the perspective of animal moves
            speeds_eye_playback[0] = 0
            speeds_eye_playback = thr(speeds_eye_playback, speed_thr)
            optics_playback = calculate_OF(rs=speeds_eye_playback, img_VS=img_VS_playback, mode='no_RF')


            # Array of distance travelled from each trial start (m)
            distance_arr_playback, _ = create_speed_arr(img_VS_playback['EyeZ'], depth_list, stim_dict_playback, mode='sort_by_depth',
                                                protocol='fix_length', blank_period=0, frame_rate=frame_rate)
            for idepth in range(distance_arr_playback.shape[0]):
                for itrial in range(distance_arr_playback.shape[1]):
                    distance_arr_playback[idepth, itrial, :] = distance_arr_playback[idepth, itrial, :] - distance_arr_playback[idepth, itrial, 0]



        # Get PSTH for all depth neurons
        for roi in depth_neurons:
            binned_stats = get_PSTH(values = dffs_original, 
            dffs=dffs_original,  
            depth_list=depth_list, 
            stim_dict=stim_dict_original,
            roi=roi,
            distance_arr=distance_arr_original, 
            distance_bins=60,
            is_trace=True)
            PSTH_ave = np.nanmean(binned_stats['binned_yrr'], axis=1).flatten()

            all_neurons_psth.append(PSTH_ave)
        
        all_neurons_preferred_depths.append(np.array(np.exp(gaussian_depth_original.x0_logged.values.astype('float32'))).tolist())
        print('Finish appending for this session.')
print('Finish appending for ALL sessions.')

all_neurons_psth = np.array(all_neurons_psth)
all_neurons_preferred_depths = np.array([j for i in all_neurons_preferred_depths for j in i])
np.save(save_root+'all_neurons_psth_old2.npy',all_neurons_psth)
np.save(save_root+'all_neurons_preferred_depths_old2.npy',all_neurons_preferred_depths)

In [ ]:
save_root = '/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver'+str(VERSION)+'/close_raster_summary/'
all_neurons_psth_old2 = np.load(save_root+'all_neurons_psth.npy')
all_neurons_preferred_depth_old2 = np.load(save_root+'all_neurons_preferred_depths.npy')
np.save(save_root+'all_neurons_psth_old2.npy', all_neurons_psth_old2)
np.save(save_root+'all_neurons_preferred_depth_old2.npy', all_neurons_preferred_depth_old2)

In [ ]:
save_root = '/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver'+str(VERSION)+'/close_raster_summary/'
    
rawdata_root = Path(flz.PARAMETERS['data_root']['raw'])
root = Path(flz.PARAMETERS['data_root']['processed'])


project = 'hey2_3d-vision_foodres_20220101'
protocol = 'SpheresPermTubeReward'
V1_sessions = [['S20230116','S20230127','S20230202'],
               ['S20230116','S20230117','S20230203','S20230209','S20230216','S20230220'],
               ['S20230117','S20230126','S20230131','S20230202','S20230206','S20230210','S20230213','S20230214']]
summary_dir = root/project/'Analysis/summary/'

mouses = ['PZAH8.2h','PZAH8.2i','PZAH8.2f']
depth_list=[0.05,0.1,0.2,0.4,0.8,1.6,3.2,6.4]

playback=False
flm_sess = flz.get_flexilims_session(project_id=project)

# COMBINE PSTH FROM ALL DEPTH NEURONS OF ALL SESSIONS INTO A GIANT LIST
all_neurons_psth = []
all_neurons_preferred_depths = []
for k in range(len(V1_sessions)):
    mouse = mouses[k]
    for session in V1_sessions[k]:
        print(mouse, session)
        frame_rate = 15
        speed_thr_cal = 0.2  # m/s, threshold for running speed when calculating depth neurons
        speed_thr = 0.01  #m/s
        manual_choose_rois = True
        manually_chosen_rois = [1]
        distance_max = 6
        
        # ----- STEP1: Generate file path -----
        session_analysis_folder_original = root/project/'Analysis'/mouse/session/'SpheresPermTubeReward'
        session_protocol_folder = root/project/mouse/session/protocol
        session_analysis_folder = root/project/'Analysis'/mouse/session/protocol
        (
        _,
        _,
        _,
        suite2p_folder,
        _,
        ) = generate_file_folders(
            project=project,
            mouse=mouse,
            session=session,
            protocol=protocol,
            all_protocol_recording_entries=None, 
            recording_no=0, 
            flexilims_session=None
        )
        
        # Load files
        # Load suite2p files
        ops = np.load(suite2p_folder/'ops.npy', allow_pickle=True)
        ops = ops.item()
        iscell = np.load(suite2p_folder/'iscell.npy', allow_pickle=True)[:, 0]

        # All_rois
        which_rois = (np.arange(iscell.shape[0]))[iscell.astype('bool')]

        # dffs_ast
        dffs_ast_all = np.load(session_protocol_folder/'plane0/dffs_ast_all.npy')

        # Load stim structure file
        with open(session_protocol_folder/'plane0/img_VS_all.pickle', 'rb') as handle:
            img_VS_all= pickle.load(handle)
        with open(session_protocol_folder/'plane0/stim_dict_all.pickle', 'rb') as handle:
            stim_dict_all = pickle.load(handle)
        with open(session_analysis_folder/'plane0/gaussian_depth_tuning_fit_new_0.5.pickle', 'rb') as handle:
            gaussian_depth = pickle.load(handle)
            
        # Load depth neurons
        depth_neurons = np.load(session_analysis_folder/'plane0/depth_neurons.npy')
        
        # PARAMS PROCESSING
        # Speed array
        # Running speed is thresholded with a small threshold to get rid of non-zero values (default threshold 0.01)
        # Closeloop
        speeds_original = img_VS_all.MouseZ.diff() / img_VS_all.HarpTime.diff() # with no playback. EyeZ and MouseZ should be the same.
        speeds_original[0] = 0
        speeds_original = thr(speeds_original, speed_thr)
        speed_arr_original, _ = create_speed_arr(speeds_original, depth_list, stim_dict_all, mode='sort_by_depth', protocol='fix_length',
                                        blank_period=0, frame_rate=frame_rate)
        speed_arr_mean_original = np.nanmean(speed_arr_original,axis=1)
        speed_arr_noblank_original,_ = create_speed_arr(speeds_original, depth_list, stim_dict_all, mode='sort_by_depth', protocol='fix_length', blank_period=0, frame_rate=frame_rate)
        speed_arr_noblank_mean_original = np.nanmean(speed_arr_noblank_original,axis=1)
        speed_arr_blank_original,_ = create_speed_arr(speeds_original, depth_list, stim_dict_all, mode='sort_by_depth', protocol='fix_length', isStim=False, blank_period=0, frame_rate=frame_rate)
        frame_num_pertrial_max_original = speed_arr_noblank_original.shape[2]
        total_trials_original = speed_arr_noblank_original.shape[1]
        optics_original = calculate_OF(rs=speeds_original, img_VS=img_VS_all, mode='no_RF')

        # Array of distance travelled from each trial start (m)
        distance_arr_original, _ = create_speed_arr(img_VS_all['EyeZ'], depth_list, stim_dict_all, mode='sort_by_depth',
                                            protocol='fix_length', blank_period=0, frame_rate=frame_rate)
        for idepth in range(distance_arr_original.shape[0]):
            for itrial in range(distance_arr_original.shape[1]):
                distance_arr_original[idepth, itrial, :] = distance_arr_original[idepth, itrial, :] - distance_arr_original[idepth, itrial, 0]

        # # Openloop
        # if playback:
        #     speeds_playback = img_VS_playback.MouseZ.diff() / img_VS_playback.HarpTime.diff() # with no playback. EyeZ and MouseZ should be the same.
        #     speeds_playback[0] = 0
        #     speeds_playback = thr(speeds_playback, speed_thr)
        #     speed_arr_playback, _ = create_speed_arr(speeds_playback, depth_list, stim_dict_playback, mode='sort_by_depth', protocol='fix_length',
        #                                     blank_period=0, frame_rate=frame_rate)
        #     speed_arr_mean_playback = np.nanmean(speed_arr_playback,axis=1)
        #     speed_arr_noblank_playback,_ = create_speed_arr(speeds_playback, depth_list, stim_dict_playback, mode='sort_by_depth', protocol='fix_length', blank_period=0, frame_rate=frame_rate)
        #     speed_arr_noblank_mean_playback = np.nanmean(speed_arr_noblank_playback,axis=1)
        #     speed_arr_blank_playback,_ = create_speed_arr(speeds_playback, depth_list, stim_dict_playback, mode='sort_by_depth', protocol='fix_length', isStim=False, blank_period=0, frame_rate=frame_rate)
        #     frame_num_pertrial_max_playback = speed_arr_noblank_playback.shape[2]
        #     total_trials_playback = speed_arr_noblank_playback.shape[1]

        #     speeds_eye_playback = img_VS_playback.EyeZ.diff() / img_VS_playback.HarpTime.diff()  # EyeZ is how the perspective of animal moves
        #     speeds_eye_playback[0] = 0
        #     speeds_eye_playback = thr(speeds_eye_playback, speed_thr)
        #     optics_playback = calculate_OF(rs=speeds_eye_playback, img_VS=img_VS_playback, mode='no_RF')


        #     # Array of distance travelled from each trial start (m)
        #     distance_arr_playback, _ = create_speed_arr(img_VS_playback['EyeZ'], depth_list, stim_dict_playback, mode='sort_by_depth',
        #                                         protocol='fix_length', blank_period=0, frame_rate=frame_rate)
        #     for idepth in range(distance_arr_playback.shape[0]):
        #         for itrial in range(distance_arr_playback.shape[1]):
        #             distance_arr_playback[idepth, itrial, :] = distance_arr_playback[idepth, itrial, :] - distance_arr_playback[idepth, itrial, 0]

        # Get PSTH for all depth neurons
        for roi in depth_neurons:
            binned_stats = get_PSTH(values = dffs_ast_all, 
            dffs=dffs_ast_all,  
            depth_list=depth_list, 
            stim_dict=stim_dict_all,
            roi=roi,
            distance_arr=distance_arr_original, 
            distance_bins=60,
            is_trace=True)
            PSTH_ave = np.nanmean(binned_stats['binned_yrr'], axis=1).flatten()

            all_neurons_psth.append(PSTH_ave)
        
        all_neurons_preferred_depths.append(np.array(np.exp(gaussian_depth.x0_logged.values.astype('float32'))).tolist())
        print('Finish appending for this session.')
print('Finish appending for ALL sessions.')
all_neurons_psth = np.array(all_neurons_psth)
all_neurons_preferred_depths = np.array([j for i in all_neurons_preferred_depths for j in i])
np.save(save_root+'all_neurons_psth_new3.npy',all_neurons_psth)
np.save(save_root+'all_neurons_preferred_depths_new3.npy',all_neurons_preferred_depths)

In [ ]:

psth_old2 = np.load(save_root+'all_neurons_psth_old2.npy')

preferred_depth_old2 = np.load(save_root+'all_neurons_preferred_depth_old2.npy')

psth_new3 = np.load(save_root+'all_neurons_psth_new3.npy')

preferred_depth_new3 = np.load(save_root+'all_neurons_preferred_depths_new3.npy')

psth_all = np.concatenate((psth_old2,psth_new3), axis=1)
preferred_depth_all = np.concatenate((preferred_depth_old2,preferred_depth_new3))

# np.save(save_root+'all_neurons_psth_5.npy',psth_all)
# np.save(save_root+'all_neurons_preferred_depth_5.npy',preferred_depth_all)

In [ ]:
# -- FIG 1: Raster plot of all neurons -- 
save_root = '/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver'+str(VERSION)+'/close_raster_summary/'
all_neurons_psth = np.load(save_root+'all_neurons_psth_old2.npy')
all_neurons_preferred_depths = np.load(save_root+'all_neurons_preferred_depth_old2.npy')

# Colormap
N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(1, 1, N)
vals[:, 1] = np.linspace(1, 0, N)
vals[:, 2] = np.linspace(1, 0, N)
WhRdcmap = ListedColormap(vals)

# Sort all neurons based on their preferred depth
inds = all_neurons_preferred_depths.argsort()
sortedPSTH_old2 = all_neurons_psth[inds,:]

# Normalize each neuron 
sortedPSTH_normed_old2 = (sortedPSTH_old2 - ((np.nanmin(sortedPSTH_old2, axis=1)).reshape(-1,1))) \
    /((np.nanmax(sortedPSTH_old2, axis=1)).reshape(-1,1)-(np.nanmin(sortedPSTH_old2, axis=1)).reshape(-1,1))
sortedPSTH_normed_old2 = np.delete(sortedPSTH_normed_old2, [4514, 7519], axis=0) # delete two cells that have negative response, get rid of these two cells from depth neurons




all_neurons_psth = np.load(save_root+'all_neurons_psth_new3.npy')
all_neurons_preferred_depths = np.load(save_root+'all_neurons_preferred_depths_new3.npy')

# Sort all neurons based on their preferred depth
inds = all_neurons_preferred_depths.argsort()
sortedPSTH_new3 = all_neurons_psth[inds,:]

# Normalize each neuron 
sortedPSTH_normed_new3 = (sortedPSTH_new3 - ((np.nanmin(sortedPSTH_new3, axis=1)).reshape(-1,1))) \
    /((np.nanmax(sortedPSTH_new3, axis=1)).reshape(-1,1)-(np.nanmin(sortedPSTH_new3, axis=1)).reshape(-1,1))
    
    
    
    
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
depth_list_old2 = [0.06, 0.19, 0.6, 1.9, 6]
depth_list_new3 = [0.05,0.1,0.2,0.4,0.8,1.6,3.2,6.4]
plt.imshow(sortedPSTH_normed_old2, aspect='auto', cmap=WhRdcmap)
plt.xticks(np.arange(sortedPSTH_old2.shape[1])[30::60],(np.array(depth_list_old2)*100).astype('int'), fontsize=15, rotation = 30)
plt.yticks(fontsize=15)
plt.ylabel('Neuron number', fontsize=15)
plt.xlabel('Corridor depth (cm)', fontsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=12)
despine()

plt.subplot(1,2,2)
plt.imshow(sortedPSTH_normed_new3, aspect='auto', cmap=WhRdcmap)
plt.xticks(np.arange(sortedPSTH_new3.shape[1])[30::60],(np.array(depth_list_new3)*100).astype('int'), fontsize=15, rotation = 30)
plt.yticks(fontsize=15)
# plt.ylabel('Neuron number', fontsize=15)
plt.xlabel('Corridor depth (cm)', fontsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=12)
cbar.ax.get_yaxis().labelpad = 20
cbar.ax.set_ylabel('Normalized \u0394F/F', fontsize = 15, rotation = 270)
despine()
plt.tight_layout(pad=1)
plt.savefig(save_root+'all_depth_neurons_raster_5.pdf')
